# Sistemas Inteligentes 2021/2022

## Mini-projeto 2: Quadrados Latinos


## Grupo: 42

### Elementos do Grupo

Número:  53687       Nome:   Ariana Dias  
Número:  51127       Nome:   Luís Ferreirinha


_o 3º colega não participou em nada_

(Nota: Neste relatório pode adicionar as células de texto e código que achar necessárias.)

## Representação de variáveis, domínios, vizinhos e restrições

O problema dos quadrados latinos pode ser representado como uma matriz $n \times n$ em que cada linha e coluna contêm exatmente um número de $1$ a $n$.  

Então para caracterizar este CSP em python obtamos pelas seguintes escolhas:  

- Variáveis:  
  
    Cada variável no nosso problema vai representar o valor atribuiado a um elemento da matriz.  
    Então em python representamos o conjunto das variáveis como uma lista das coordenadas de cada
    elemento na matriz:    
    -  var $=[(i_1,j_1),...,(i_n,j_n)]$  
<br>
- Domínios:  
  
    Neste problema cada variável pode tomar um valor de $1,...,n$, exceto de houver condições iniciais.
    Portanto em python decidimos utilizar um dicionário, onde as chaves são as variáveis e estas correspondem a uma lista de inteiros contento os valores possíveis para aquela variável:
    
    - dominio $=\{(i_1,j_1) = [1,..,n], ...\}$  
<br> 
- Vizinhos:  

    Uma variável vai ser vizinha de outra de estiver na mesma linha ou coluna que essa variável.
    Em python definimos os vizinhos como um dicionário onde cada variável é uma chave e corresponde a uma lista de variáveis que são suas vizinhas:  
    
    - vizinhos $=\{(i_k,i_m) : [(i_1,j_m),...,(i_n,j_m),(i_k,j_1),...,(i_k,j_n)], \, ...  \, \}$  
<br>
- Restrições:  

    A restrição principal deste problema é a impossibilidade de haver repetições de um número
    ao longo de uma coluna ou linha, ou seja tem de ser diferente de todos os seus vizinhos.
    Em python esta restrição é representada pela comparação dos valores de duas variáveis vizinhas:  

    -  ```python
       valor_variavel_1 != valor_variavel_2
       ```
    <br>
    Como também temos a variação do Futoshiki, temos de definir outras restrições para esse problema.  

    A ideia principal é que duas variáveis adjacentes podem agora ter a restrição adicional de o valor de uma ser maior ou menor que o valor da outra. Então para representar o mapeamento destas novas restrições usamos um dicionário onde as chaves são um tuplos contendos as duas variáveis em questão e vão corresponde a uma função que verifica se os valores destas são menores ou maiores conforme a restrição.

    - desigualdades $=\{((i_k,j_m),(i_k,j_{m+1})) : maior\, \, \text{ou} \, \, menor \}$  

    Cada uma destas funções vair comparar os valores das variavéis conforme o seu nome.  

    - ```python 
      def maior(a,b): return a > b
      ```

    - ```python 
      def menor(a,b): return a < b
      ```


 

## Formulação do problema

In [1]:
from csp import *

# Funcoes restricao
def maior(x,y):
    return x > y

def menor(x,y):
    return x < y

def quadrado_latino(n=3, initial={}, desigualdades={}) -> CSP:
    """Gera um CSP para o problema do futoshiki

    Args:
        n (int, optional): Dimensao da matriz do problema. Defaults to 3.
        initial (dict, optional): Valores iniciais para cada um dos elementos da matriz. Defaults to {}.
        desigualdades (dict, optional): Restrições adicionais do problema futoshiki. Defaults to {}.

    Returns:
        CSP: Objeto CSP para o problema do futoshiki
    """    
    
    # definicao das variaveis, as coordenadas x,y vão de 0 a n-1
    variaveis = [(i,j) for i in range(0,n) for j in range(0,n)]

    # Valores possiveis para as variaveis dada a dimensao n do problema
    conjunto = [i for i in range(1,n+1)]
    
    # Iniciamos os dominios com os valores iniciais dados para as variaveis
    dominios = initial

    # Diferenca dos dois conjuntos vai resultar nas variaveis nao preenchidas
    for item in set(variaveis - initial.keys()):
        dominios[item] = conjunto
    
    # Condicao que determina de duas variaveis sao vizinhas, verifica se tem alguma das coordendas x ou y iguais
    condicao_vizinhanca = lambda x,y : (x[0] == y[0] and x[1] != y[1]) or (x[0] != y[0] and x[1] == y[1])

    # Para cada variavel vamos atribuir uma lista resultante de filtrar a lista de todas as variaveis
    # com a condicao de que as variaveis sejam vizinhas da variavel chave
    vizinhos = {var : list(filter(lambda x: condicao_vizinhanca(x,var), variaveis)) for var in variaveis}

    # Restricoes do futoshiki, para o utilizador nao ter de escrever as restricoes de desigualdade nas duas direcoes
    # Vamos criar um dicionario com essas restricoes de forma bi-direcional
    restricoes_desigualdades = desigualdades.copy()
    for var, restricao in desigualdades.items():
        if restricao.__name__ == "maior":
            restricoes_desigualdades[(var[1],var[0])] = menor
        else:
            restricoes_desigualdades[(var[1],var[0])] = maior

    def restricoes(X, a, Y, b):
        # Restricoes futoshiki
        # Vamos verificar se a var X tem alguma condição de desigualdade com outra var Y
        if (X,Y) in restricoes_desigualdades.keys():
            if not restricoes_desigualdades[(X,Y)](a,b):
                return False
        # Restricao normal quadrado latino
        # Verifica se sao vizinhos
        if Y in vizinhos[X]:
            return  a != b
        return True

    return CSP(variaveis, dominios, vizinhos, restricoes)

## Criação do problema do quadrado latino simples

Mostrem que o código está a funcionar, construindo um problema de quadrado latino *4x4*, imprimindo as variáveis, domínios iniciais, e vizinhos. Adicione os comentários necessários. Mostre como podemos criar um puzzle com quadrados já preenchidos, e qual o impacto que isso tem nas variáveis, domínios iniciais, e vizinhos.

Para criar um problema quadrado latino 4x4 sem valores iniciais para as variáveis basta executar

In [2]:
q4 = quadrado_latino(4)

Tendo o problema definido, podemos agora ver as variáveis, domínios e os vizinhos deste problema

In [3]:
print(f"Variáveis = {q4.variables}\n")
print(f"Domínios = {q4.domains}\n")
print(f"Vizinhos = {q4.neighbors}\n")

Variáveis = [(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3), (2, 0), (2, 1), (2, 2), (2, 3), (3, 0), (3, 1), (3, 2), (3, 3)]

Domínios = {(0, 1): [1, 2, 3, 4], (1, 2): [1, 2, 3, 4], (2, 1): [1, 2, 3, 4], (3, 1): [1, 2, 3, 4], (0, 2): [1, 2, 3, 4], (2, 2): [1, 2, 3, 4], (1, 0): [1, 2, 3, 4], (3, 2): [1, 2, 3, 4], (1, 3): [1, 2, 3, 4], (0, 0): [1, 2, 3, 4], (1, 1): [1, 2, 3, 4], (0, 3): [1, 2, 3, 4], (2, 0): [1, 2, 3, 4], (3, 0): [1, 2, 3, 4], (2, 3): [1, 2, 3, 4], (3, 3): [1, 2, 3, 4]}

Vizinhos = {(0, 0): [(0, 1), (0, 2), (0, 3), (1, 0), (2, 0), (3, 0)], (0, 1): [(0, 0), (0, 2), (0, 3), (1, 1), (2, 1), (3, 1)], (0, 2): [(0, 0), (0, 1), (0, 3), (1, 2), (2, 2), (3, 2)], (0, 3): [(0, 0), (0, 1), (0, 2), (1, 3), (2, 3), (3, 3)], (1, 0): [(0, 0), (1, 1), (1, 2), (1, 3), (2, 0), (3, 0)], (1, 1): [(0, 1), (1, 0), (1, 2), (1, 3), (2, 1), (3, 1)], (1, 2): [(0, 2), (1, 0), (1, 1), (1, 3), (2, 2), (3, 2)], (1, 3): [(0, 3), (1, 0), (1, 1), (1, 2), (2, 3), (3, 3)], (2, 0): [(0, 0), 

Para criar-mos um problema com um quadrado semi-preenchido temos primeiro de definir quais os elementos da matriz que vão estar preenchidos e qual o seu valor. Para fazer-mos isso podemos definir um dicionário com essa informação da seguinte forma:

In [4]:
valores_preenchidos = {(0,0) : [1], (1,1) : [3], (2,2) : [2]}

Depois basta passar esse dicionário como segundo argumento à função quadrado_latino

In [5]:
q4_semi = quadrado_latino(n=4, initial=valores_preenchidos)

O facto de o quadrado ser semi preenchido apenas vai afetar os domínios iniciais, fazendo com que o dominio inicial das variáveis já com valores atribuidos seja reduzido esses valores.

In [6]:
print(f"Variáveis = {q4_semi.variables}\n")
print(f"Domínios = {q4_semi.domains}\n")
print(f"Vizinhos = {q4_semi.neighbors}\n")

Variáveis = [(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3), (2, 0), (2, 1), (2, 2), (2, 3), (3, 0), (3, 1), (3, 2), (3, 3)]

Domínios = {(0, 0): [1], (1, 1): [3], (2, 2): [2], (0, 1): [1, 2, 3, 4], (1, 2): [1, 2, 3, 4], (2, 1): [1, 2, 3, 4], (3, 1): [1, 2, 3, 4], (0, 3): [1, 2, 3, 4], (2, 0): [1, 2, 3, 4], (3, 0): [1, 2, 3, 4], (2, 3): [1, 2, 3, 4], (0, 2): [1, 2, 3, 4], (3, 3): [1, 2, 3, 4], (1, 0): [1, 2, 3, 4], (3, 2): [1, 2, 3, 4], (1, 3): [1, 2, 3, 4]}

Vizinhos = {(0, 0): [(0, 1), (0, 2), (0, 3), (1, 0), (2, 0), (3, 0)], (0, 1): [(0, 0), (0, 2), (0, 3), (1, 1), (2, 1), (3, 1)], (0, 2): [(0, 0), (0, 1), (0, 3), (1, 2), (2, 2), (3, 2)], (0, 3): [(0, 0), (0, 1), (0, 2), (1, 3), (2, 3), (3, 3)], (1, 0): [(0, 0), (1, 1), (1, 2), (1, 3), (2, 0), (3, 0)], (1, 1): [(0, 1), (1, 0), (1, 2), (1, 3), (2, 1), (3, 1)], (1, 2): [(0, 2), (1, 0), (1, 1), (1, 3), (2, 2), (3, 2)], (1, 3): [(0, 3), (1, 0), (1, 1), (1, 2), (2, 3), (3, 3)], (2, 0): [(0, 0), (1, 0), (2, 1), (2, 2), (2,

E podemos resolver utilizando a função backtracking_seach da biblioteca CSP

In [7]:
sol_q4_semi = backtracking_search(q4_semi)

E como a função devolve um dicionário com os valores solução afetados às variáveis podemos imprimir este

In [8]:
print(sol_q4_semi)

{(0, 0): 1, (0, 1): 2, (0, 2): 3, (0, 3): 4, (1, 0): 2, (1, 1): 3, (1, 2): 4, (1, 3): 1, (2, 0): 4, (2, 1): 1, (2, 2): 2, (2, 3): 3, (3, 0): 3, (3, 1): 4, (3, 2): 1, (3, 3): 2}


Resolva o problema com o backtracking sem inferencia, com inferencia, e com uma heurística.

Vamos agora resolver um problema de maior dimensão pois temos como objetivo comparar os tempos de procura quando são aplicados diferentes algoritmos durante a procura (ou antes caso do pré processamento AC3).  
Para isso vamos definir uma função wrapper que vai calcular o tempo de execução da pesquisa.

In [9]:
import timeit

def search_time(func, *args, **kwargs):
    """Calcula o tempo de execução de uma função e devolve
    o seu output e este tempo

    Args:
        func: função a ser cronometrada
    """
    start = timeit.default_timer()
    sol = func(*args, **kwargs)
    stop = timeit.default_timer()
    return sol, (stop-start)

# Guardar os dados e solucoes para analisar mais tarde
tempos = {}
solucoes = {}

- Definição do problema

In [10]:
def board_to_dict(board) -> dict:
    """Coverte uma matriz quadrada de inteiros para um dicionario
    contendo as posicoes dos inteiros nao nulos
    """
    n = len(board)
    iniciais = {}
    for j in range(n):
        for i in range(n):
            x = board[j][i]
            if x != 0:
                iniciais[(i,j)] = [x]
    return iniciais

In [11]:
n = 7

board = [[6,0,0,0,0,0,0],
         [0,6,0,1,0,0,3],
         [2,0,7,0,0,0,0],
         [4,0,0,2,0,0,1],
         [0,3,1,5,6,0,0],
         [1,0,0,7,0,2,4],
         [0,0,0,0,0,0,7]]


p_quadrado_latino = quadrado_latino(n, initial=board_to_dict(board))

- Backtracking sem inferência

In [12]:
# Guardar solucao
sol1, tempo1 = search_time(backtracking_search, p_quadrado_latino)
# Guardar resultados
tempos['SemInferência'] = tempo1
solucoes['SemInferência'] = sol1
# Imprimir resultados
print(f"Tempo médio de execução: {tempo1:.6f} s")
print('Assignment (SemInferência) = ',sol1)

Tempo médio de execução: 17.008735 s
Assignment (SemInferência) =  {(0, 0): 6, (0, 1): 5, (0, 2): 2, (0, 3): 4, (0, 4): 7, (0, 5): 1, (0, 6): 3, (1, 0): 1, (1, 1): 6, (1, 2): 4, (1, 3): 7, (1, 4): 3, (1, 5): 5, (1, 6): 2, (2, 0): 2, (2, 1): 4, (2, 2): 7, (2, 3): 3, (2, 4): 1, (2, 5): 6, (2, 6): 5, (3, 0): 4, (3, 1): 1, (3, 2): 3, (3, 3): 2, (3, 4): 5, (3, 5): 7, (3, 6): 6, (4, 0): 7, (4, 1): 2, (4, 2): 1, (4, 3): 5, (4, 4): 6, (4, 5): 3, (4, 6): 4, (5, 0): 3, (5, 1): 7, (5, 2): 5, (5, 3): 6, (5, 4): 4, (5, 5): 2, (5, 6): 1, (6, 0): 5, (6, 1): 3, (6, 2): 6, (6, 3): 1, (6, 4): 2, (6, 5): 4, (6, 6): 7}


- Backtracking com inferência Foward-Checking

In [13]:
# Resolver e guardar solucao
sol2, tempo2 = search_time(backtracking_search, p_quadrado_latino, inference=forward_checking)
# Guardar resultados
tempos['FRWDchecking'] = tempo2
solucoes['FRWDchecking'] = sol2
# Imprimir resultados
print(f"Tempo médio de execução: {tempo2:.6f} s")
print('Assignment (FRWDchecking) = ',sol2)

Tempo médio de execução: 0.002879 s
Assignment (FRWDchecking) =  {(0, 0): 6, (0, 1): 5, (0, 2): 2, (0, 3): 4, (0, 4): 7, (0, 5): 1, (0, 6): 3, (1, 0): 1, (1, 1): 6, (1, 2): 4, (1, 3): 7, (1, 4): 3, (1, 5): 5, (1, 6): 2, (2, 0): 2, (2, 1): 4, (2, 2): 7, (2, 3): 3, (2, 4): 1, (2, 5): 6, (2, 6): 5, (3, 0): 4, (3, 1): 1, (3, 2): 3, (3, 3): 2, (3, 4): 5, (3, 5): 7, (3, 6): 6, (4, 0): 7, (4, 1): 2, (4, 2): 1, (4, 3): 5, (4, 4): 6, (4, 5): 3, (4, 6): 4, (5, 0): 3, (5, 1): 7, (5, 2): 5, (5, 3): 6, (5, 4): 4, (5, 5): 2, (5, 6): 1, (6, 0): 5, (6, 1): 3, (6, 2): 6, (6, 3): 1, (6, 4): 2, (6, 5): 4, (6, 6): 7}


- Backtracking com inferência MAC

In [14]:
# Resolver e guardar solucao
sol3, tempo3 = search_time(backtracking_search, p_quadrado_latino, inference=mac)
# Guarda resultados
tempos['MAC'] = tempo3
solucoes['MAC'] = sol3
# Imprimir resultados
print(f"Tempo médio de execução: {tempo3:.6f}  s")
print('Assignment (MAC) = ', sol3)

Tempo médio de execução: 0.001931  s
Assignment (MAC) =  {(0, 0): 6, (0, 1): 5, (0, 2): 2, (0, 3): 4, (0, 4): 7, (0, 5): 1, (0, 6): 3, (1, 0): 1, (1, 1): 6, (1, 2): 4, (1, 3): 7, (1, 4): 3, (1, 5): 5, (1, 6): 2, (2, 0): 2, (2, 1): 4, (2, 2): 7, (2, 3): 3, (2, 4): 1, (2, 5): 6, (2, 6): 5, (3, 0): 4, (3, 1): 1, (3, 2): 3, (3, 3): 2, (3, 4): 5, (3, 5): 7, (3, 6): 6, (4, 0): 7, (4, 1): 2, (4, 2): 1, (4, 3): 5, (4, 4): 6, (4, 5): 3, (4, 6): 4, (5, 0): 3, (5, 1): 7, (5, 2): 5, (5, 3): 6, (5, 4): 4, (5, 5): 2, (5, 6): 1, (6, 0): 5, (6, 1): 3, (6, 2): 6, (6, 3): 1, (6, 4): 2, (6, 5): 4, (6, 6): 7}


- Backtracking com pré-processamento AC3

In [15]:
# Aplicar pre-processamento
p_quadrado_AC3 = AC3(p_quadrado_latino)
# Resolver e guardar solucao
sol4, tempo4 = search_time(backtracking_search, p_quadrado_AC3)
# Guarda resultados
tempos['AC3'] = tempo4
solucoes['AC3'] = sol4
# Imprimir resultados
print(f"Tempo médio de execução: {tempo4:.6f}  s")
print('Assignment (QuadradoLatinoComAC3) = ', sol4)

Tempo médio de execução: 0.001106  s
Assignment (QuadradoLatinoComAC3) =  {(0, 0): 6, (0, 1): 5, (0, 2): 2, (0, 3): 4, (0, 4): 7, (0, 5): 1, (0, 6): 3, (1, 0): 1, (1, 1): 6, (1, 2): 4, (1, 3): 7, (1, 4): 3, (1, 5): 5, (1, 6): 2, (2, 0): 2, (2, 1): 4, (2, 2): 7, (2, 3): 3, (2, 4): 1, (2, 5): 6, (2, 6): 5, (3, 0): 4, (3, 1): 1, (3, 2): 3, (3, 3): 2, (3, 4): 5, (3, 5): 7, (3, 6): 6, (4, 0): 7, (4, 1): 2, (4, 2): 1, (4, 3): 5, (4, 4): 6, (4, 5): 3, (4, 6): 4, (5, 0): 3, (5, 1): 7, (5, 2): 5, (5, 3): 6, (5, 4): 4, (5, 5): 2, (5, 6): 1, (6, 0): 5, (6, 1): 3, (6, 2): 6, (6, 3): 1, (6, 4): 2, (6, 5): 4, (6, 6): 7}


- Backtracking com Heurística MRV

In [16]:
# Resolver e guardar
sol5, tempo5 = search_time(backtracking_search, p_quadrado_latino, select_unassigned_variable = mrv)
# Guarda resultados
tempos['MRV'] = tempo5
solucoes['MRV'] = sol5
# Imprimir resultados
print(f"Tempo médio de execução: {tempo5:.6f} s")
print('Assignment (QuadradoLatinoComHeuristica) = ', sol5)

Tempo médio de execução: 0.003627 s
Assignment (QuadradoLatinoComHeuristica) =  {(0, 6): 3, (1, 5): 5, (3, 0): 4, (1, 3): 7, (0, 4): 7, (2, 5): 6, (5, 2): 5, (4, 5): 3, (5, 0): 3, (0, 1): 5, (2, 2): 7, (4, 1): 2, (3, 4): 5, (3, 3): 2, (1, 2): 4, (3, 5): 7, (2, 6): 5, (4, 3): 5, (1, 1): 6, (5, 6): 1, (6, 6): 7, (2, 1): 4, (6, 0): 5, (2, 4): 1, (3, 6): 6, (0, 2): 2, (6, 5): 4, (2, 3): 3, (1, 6): 2, (6, 4): 2, (2, 0): 2, (4, 4): 6, (3, 2): 3, (1, 0): 1, (0, 5): 1, (6, 1): 3, (5, 4): 4, (4, 6): 4, (4, 0): 7, (1, 4): 3, (4, 2): 1, (0, 0): 6, (5, 1): 7, (3, 1): 1, (0, 3): 4, (5, 5): 2, (5, 3): 6, (6, 3): 1, (6, 2): 6}


- Backtracking com Heurística MRV, Inferência foward_checking e pré-processamento AC3

In [17]:
# Pre processamento
p_quadrado_AC3 = AC3(p_quadrado_latino)
# Resolver e guardar
sol6, tempo6 = search_time(backtracking_search, p_quadrado_AC3, select_unassigned_variable=mrv, inference=forward_checking)
# Guarda resultados
tempos['AC3_MVR_MAC'] = tempo6
solucoes['AC3_MVR_MAC'] = sol6
# Imprimir resultados
print(f"Tempo médio de execução: {tempo6:.6f} s")
print('Assignment (QuadradoLatinoHeuristicaInferenciaAC3) = ', sol6)

Tempo médio de execução: 0.002556 s
Assignment (QuadradoLatinoHeuristicaInferenciaAC3) =  {(2, 5): 6, (4, 2): 1, (1, 5): 5, (2, 4): 1, (3, 0): 4, (2, 1): 4, (4, 3): 5, (1, 2): 4, (5, 3): 6, (1, 4): 3, (3, 4): 5, (5, 5): 2, (6, 0): 5, (5, 0): 3, (0, 2): 2, (0, 5): 1, (6, 5): 4, (6, 4): 2, (3, 3): 2, (3, 1): 1, (0, 1): 5, (1, 1): 6, (5, 6): 1, (0, 4): 7, (5, 4): 4, (4, 0): 7, (0, 3): 4, (4, 5): 3, (6, 1): 3, (6, 3): 1, (3, 5): 7, (6, 6): 7, (1, 6): 2, (3, 2): 3, (6, 2): 6, (1, 3): 7, (0, 0): 6, (1, 0): 1, (2, 0): 2, (2, 3): 3, (5, 1): 7, (4, 4): 6, (2, 2): 7, (4, 6): 4, (0, 6): 3, (4, 1): 2, (2, 6): 5, (3, 6): 6, (5, 2): 5}


#### Análise dos tempos de execução

In [18]:
import pandas as pd
temposSorted = dict(sorted(tempos.items(), key=lambda x:x[1]))
pd.DataFrame(temposSorted.values() ,index = temposSorted.keys(), columns= ["Tempos (s)"])

,Tempos (s)
AC3,0.001106
MAC,0.001931
AC3_MVR_MAC,0.002556
FRWDchecking,0.002879
MRV,0.003627
SemInferência,17.008735


In [26]:
pd.DataFrame(solucoes).T.drop_duplicates()

0                    1        ...  5        6                  
               0  1  2  3  4  5  6  0  1  2  ...  4  5  6  0  1  2  3  4  5  6
SemInferência  6  5  2  4  7  1  3  1  6  4  ...  4  2  1  5  3  6  1  2  4  7

[1 rows x 49 columns]

Comparando os tempos obtidos nas diferentes configurações do algoritmo backtracking search pudemos concluir que para um problema de baixa dimensão e com poucas restrições como é o caso deste (quadrado latino 8x8 sem casas inicializadas) nem sempre a aplicação de uma heurística ou de uma inferência vai reduzir o tempo de procura. Isto deve-se ao facto de estes algoritmos que correm durante a procura fazerem checks adicionais em cada iteração do backtracking search, resultando no fim em menos iterações do backtracking search mas mais demorosas. Num problema de grande complexidade isto ajuda a reduzir o domínio das variavéis e a convergir numa solução mais rápidamente, mas para este problema simples acaba-se por demorar mais tempo a fazer estes checks do que a experimentar combinações de acordo com a ordem no domínio das variáveis. Mesmo assim têm o seu mérito pois a inferência foward checking demorou menos tempo que o backtracking sem inferência e os outros tiveram tempos semelhantes.
O pré-processamento AC3 como trata de reduzir o domínio das varíaveis antes da execução da procura obtem o melhor tempo, pois este simplifica o problema antes de ser resolvido.  
<br>
Podemos notar também que as soluções obtidas por todas as configurações de procura foram iguais.

## Visualização do problema

In [27]:
def visualizacao_csp_quadrado_latino(n, problema_csp, solucao):
    """Gera uma visualização do estado inicial e da solução de um problema de quadrados
    latinos de dimensão n
    
    Args:
        n (int): Dimensão do problema
        problema_csp (CSP): Object CSP do problema em questão
        solucao (dict): Solução do problema
    """    
    
    # Construir matriz inicial com os valores do problema
    estado_inicial = [[0 for y in range(n)] for x in range(n)]
    for var, dom in problema_csp.domains.items():
        if len(dom) == 1:
            estado_inicial[var[1]][var[0]] = dom[0]
        
    # Construir matriz final com os valores do problema
    estado_final = [[0 for y in range(n)] for x in range(n)]
    for var, valor in solucao.items():
        estado_final[var[1]][var[0]] = valor
    
    # Construir strings que representam as matrizes
    board_inicial = ""
    board_final = ""
    for y in range(n):
        for x in range(n):
            board_inicial += str(estado_inicial[y][x]) + " "
            board_final += str(estado_final[y][x]) + " "
        board_inicial += "\n"
        board_final += "\n"
    
    print("Estado Inicial:")
    print(board_inicial)
    print("Solução:")
    print(board_final)

def visualizacao_board(board):
    """Dada uma board de um problema quadrado latino, faz
    print do output
    """
    n = len(board)
    output = ""
    for j in range(n):
        for i in range(n):
            output += str(board[j][i]) + " "
        output += "\n"
    print(output)

Podemos ver a board orignal e solução já conhecida para comparar com o obtido pelo algoritmo

In [28]:
solution = [[6,1,2,4,7,3,5],
            [5,6,4,1,2,7,2],
            [2,4,7,3,1,5,6],
            [4,7,3,2,5,6,1],
            [7,3,1,5,6,4,2],
            [1,5,6,7,3,2,4],
            [3,2,5,6,4,1,7]]

print("Antes:")
visualizacao_board(board)
print("Depois:")
visualizacao_board(solution)

Antes:
6 0 0 0 0 0 0 
0 6 0 1 0 0 3 
2 0 7 0 0 0 0 
4 0 0 2 0 0 1 
0 3 1 5 6 0 0 
1 0 0 7 0 2 4 
0 0 0 0 0 0 7 

Depois:
6 1 2 4 7 3 5 
5 6 4 1 2 7 2 
2 4 7 3 1 5 6 
4 7 3 2 5 6 1 
7 3 1 5 6 4 2 
1 5 6 7 3 2 4 
3 2 5 6 4 1 7 



Para visualizar a solução do problema anterior basta invocar a função visualizacao_quadrado_latino dando a dimensao e a solução como argumentos

In [29]:
visualizacao_csp_quadrado_latino(n, p_quadrado_latino, backtracking_search(p_quadrado_latino))

Estado Inicial:
6 0 0 0 0 0 0 
0 6 0 1 0 0 3 
2 0 7 0 0 0 0 
4 0 0 2 0 0 1 
0 3 1 5 6 0 0 
1 0 0 7 0 2 4 
0 0 0 0 0 0 7 

Solução:
6 1 2 4 7 3 5 
5 6 4 1 2 7 3 
2 4 7 3 1 5 6 
4 7 3 2 5 6 1 
7 3 1 5 6 4 2 
1 5 6 7 3 2 4 
3 2 5 6 4 1 7 



## Criação do problema Futoshiki *5x5*

Mostrem que o código está a funcionar, construindo um problema de Futoshiki *5x5*, imprimindo as variáveis, domínios iniciais, e vizinhos. Adicione os comentários necessários. Utilize o [link](https://www.futoshiki.org/) para gerar puzzles e validar a implementação.

Para criar um problema Futoshiki vamos ter definir os elementos da matriz que estão preenchidos e estabelecer as relações de desigualdade presentes no problema.

In [30]:
preenchidos5 = {(0,3) : [2], (4,3) : [3]}
desigualdades5 = {((3,0),(4,0)) : maior, ((0,1),(1,1)) : menor, ((0,2),(1,2)) : menor,\
                 ((3,3),(3,2)) : menor, ((1,4),(2,4)) : maior, ((2,4),(2,3)) : maior, \
                 ((4,4),(4,3)) : maior}

Podemos então criar um problema Futoshiki 5x5 da seguinte forma

In [31]:
futoshiki5 = quadrado_latino(5, preenchidos5, desigualdades5)

E ver as variáveis, domínios e vizinhos

In [32]:
print("Variáveis = ", futoshiki5.variables, "\n")
print("Domínios = ", futoshiki5.domains,"\n")
print("Vizinhos = ", futoshiki5.neighbors, "\n")

Variáveis =  [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4)] 

Domínios =  {(0, 3): [2], (4, 3): [3], (0, 1): [1, 2, 3, 4, 5], (2, 4): [1, 2, 3, 4, 5], (4, 0): [1, 2, 3, 4, 5], (1, 2): [1, 2, 3, 4, 5], (3, 4): [1, 2, 3, 4, 5], (0, 4): [1, 2, 3, 4, 5], (2, 1): [1, 2, 3, 4, 5], (3, 1): [1, 2, 3, 4, 5], (0, 2): [1, 2, 3, 4, 5], (2, 2): [1, 2, 3, 4, 5], (1, 0): [1, 2, 3, 4, 5], (3, 2): [1, 2, 3, 4, 5], (1, 3): [1, 2, 3, 4, 5], (4, 1): [1, 2, 3, 4, 5], (4, 4): [1, 2, 3, 4, 5], (0, 0): [1, 2, 3, 4, 5], (1, 1): [1, 2, 3, 4, 5], (2, 0): [1, 2, 3, 4, 5], (4, 2): [1, 2, 3, 4, 5], (3, 0): [1, 2, 3, 4, 5], (1, 4): [1, 2, 3, 4, 5], (2, 3): [1, 2, 3, 4, 5], (3, 3): [1, 2, 3, 4, 5]} 

Vizinhos =  {(0, 0): [(0, 1), (0, 2), (0, 3), (0, 4), (1, 0), (2, 0), (3, 0), (4, 0)], (0, 1): [(0, 0), (0, 2), (0, 3), (0, 4), (1, 1), (2, 1), (3, 1), (4, 1)], (0, 2):

E podemos resolver usando o backtracking search

In [33]:
sol_futo = backtracking_search(futoshiki5)
print("Solução:", sol_futo)

Solução: {(0, 0): 5, (0, 1): 3, (0, 2): 1, (0, 3): 2, (0, 4): 4, (1, 0): 1, (1, 1): 4, (1, 2): 2, (1, 3): 5, (1, 4): 3, (2, 0): 4, (2, 1): 5, (2, 2): 3, (2, 3): 1, (2, 4): 2, (3, 0): 3, (3, 1): 2, (3, 2): 5, (3, 3): 4, (3, 4): 1, (4, 0): 2, (4, 1): 1, (4, 2): 4, (4, 3): 3, (4, 4): 5}


Resolva o problema com o backtracking sem inferencia, com inferencia, e com uma heurística. Até que dimensão consegue resolver o problema em menos de 1 minuto?

Tal como para os quadrados latinos vamos criar um problema de maior dimensão com restrições complexas de forma a pudermos comparar melhor os tempos de execução.

In [45]:
# import timeit
#
# def avg_run_time(func, *args, **kwargs):
#     """Corre uma função n vezes medindo o tempo de execução de cada vez
#     e imprime a média dos tempos
#
#     Args:
#         func: Função a correr
#     """
#     n = 1
#     run_times = []
#     for i in range(n):
#         start = timeit.default_timer()
#         func(*args, **kwargs)
#         stop = timeit.default_timer()
#         run_times.append(stop-start)
#     return sum(run_times) / n

solucoesFutoshiki = {}
temposFutoshiki = {}

In [46]:
preenchidos7ext = {(3,2) : [2], (4,0) : [4]}
desigualdades7ext = {((1,0),(2,0)):menor, ((2,0),(3,0)):menor, ((5,0),(6,0)): maior, ((2,1),(2,0)):menor, ((5,1),(5,0)):menor, ((0,1),(1,1)):maior, ((1,1),(2,1)):maior, ((2,1),(3,1)): menor, ((4,1),(5,1)):menor, ((2,2),(2,1)): menor, ((5,2),(5,1)): maior, ((1,3),(1,2)): menor, ((2,3),(2,2)): maior, ((0,3),(1,3)):maior, ((2,3),(3,3)):menor, ((4,3),(5,3)): menor, ((0,4),(0,3)):maior,((1,4),(1,3)):menor,((6,4),(6,3)):menor, ((3,4),(4,4)):menor,((4,4),(5,4)):menor,((3,6),(3,5)):maior, ((3,5),(3,4)):menor, ((0,6),(0,5)): maior, ((0,2),(1,2)): maior}
futoshiki7ext = quadrado_latino(7, preenchidos7ext, desigualdades7ext)

- Backtracking sem inferência

In [47]:
# Guardar solucao
sol1, tempo1 = search_time(backtracking_search, futoshiki7ext)
# Guardar resultados
temposFutoshiki['SemInferência'] = tempo1
solucoesFutoshiki['SemInferência'] = sol1
# Imprimir resultados
print(f"Tempo médio de execução: {tempo1:.6f} s")
print('Assignment (SemInferência) = ',sol1)

Tempo médio de execução: 51.257967 s
Assignment (SemInferência) =  {(0, 0): 1, (0, 1): 7, (0, 2): 6, (0, 3): 4, (0, 4): 5, (0, 5): 2, (0, 6): 3, (1, 0): 2, (1, 1): 6, (1, 2): 4, (1, 3): 3, (1, 4): 1, (1, 5): 7, (1, 6): 5, (2, 0): 3, (2, 1): 2, (2, 2): 1, (2, 3): 5, (2, 4): 4, (2, 5): 6, (2, 6): 7, (3, 0): 7, (3, 1): 5, (3, 2): 2, (3, 3): 6, (3, 4): 3, (3, 5): 1, (3, 6): 4, (4, 0): 4, (4, 1): 3, (4, 2): 7, (4, 3): 1, (4, 4): 6, (4, 5): 5, (4, 6): 2, (5, 0): 6, (5, 1): 4, (5, 2): 5, (5, 3): 2, (5, 4): 7, (5, 5): 3, (5, 6): 1, (6, 0): 5, (6, 1): 1, (6, 2): 3, (6, 3): 7, (6, 4): 2, (6, 5): 4, (6, 6): 6}


- Backtracking com inferência Forward Checking

In [48]:
# Guardar solucao
sol2, tempo2 = search_time(backtracking_search, futoshiki7ext, inference=forward_checking)
# Guardar resultados
temposFutoshiki['FRWDchecking'] = tempo2
solucoesFutoshiki['FRWDchecking'] = sol2
# Imprimir resultados
print(f"Tempo médio de execução: {tempo2:.6f} s")
print('Assignment (FRWDchecking) = ',sol2)

Tempo médio de execução: 0.004150 s
Assignment (FRWDchecking) =  {(0, 0): 1, (0, 1): 7, (0, 2): 6, (0, 3): 4, (0, 4): 5, (0, 5): 2, (0, 6): 3, (1, 0): 2, (1, 1): 6, (1, 2): 4, (1, 3): 3, (1, 4): 1, (1, 5): 7, (1, 6): 5, (2, 0): 3, (2, 1): 2, (2, 2): 1, (2, 3): 5, (2, 4): 4, (2, 5): 6, (2, 6): 7, (3, 0): 7, (3, 1): 5, (3, 2): 2, (3, 3): 6, (3, 4): 3, (3, 5): 1, (3, 6): 4, (4, 0): 4, (4, 1): 3, (4, 2): 7, (4, 3): 1, (4, 4): 6, (4, 5): 5, (4, 6): 2, (5, 0): 6, (5, 1): 4, (5, 2): 5, (5, 3): 2, (5, 4): 7, (5, 5): 3, (5, 6): 1, (6, 0): 5, (6, 1): 1, (6, 2): 3, (6, 3): 7, (6, 4): 2, (6, 5): 4, (6, 6): 6}


- Backtracking com inferência MAC

In [49]:
# Guardar solucao
sol3, tempo3 = search_time(backtracking_search, futoshiki7ext, inference=mac)
# Guardar resultados
temposFutoshiki['MAC'] = tempo3
solucoesFutoshiki['MAC'] = sol3
# Imprimir resultados
print(f"Tempo médio de execução: {tempo3:.6f} s")
print('Assignment (MAC) = ',sol3)

Tempo médio de execução: 0.006409 s
Assignment (MAC) =  {(0, 0): 1, (0, 1): 7, (0, 2): 6, (0, 3): 4, (0, 4): 5, (0, 5): 2, (0, 6): 3, (1, 0): 2, (1, 1): 6, (1, 2): 4, (1, 3): 3, (1, 4): 1, (1, 5): 7, (1, 6): 5, (2, 0): 3, (2, 1): 2, (2, 2): 1, (2, 3): 5, (2, 4): 4, (2, 5): 6, (2, 6): 7, (3, 0): 7, (3, 1): 5, (3, 2): 2, (3, 3): 6, (3, 4): 3, (3, 5): 1, (3, 6): 4, (4, 0): 4, (4, 1): 3, (4, 2): 7, (4, 3): 1, (4, 4): 6, (4, 5): 5, (4, 6): 2, (5, 0): 6, (5, 1): 4, (5, 2): 5, (5, 3): 2, (5, 4): 7, (5, 5): 3, (5, 6): 1, (6, 0): 5, (6, 1): 1, (6, 2): 3, (6, 3): 7, (6, 4): 2, (6, 5): 4, (6, 6): 6}


- Backtracking com pré-processamento AC3

In [50]:
# Aplicar pre-processamento
p_quadrado_AC3 = AC3(p_quadrado_latino)
# Resolver e guardar solucao
sol4, tempo4 = search_time(backtracking_search, futoshiki7ext)
# Guarda resultados
temposFutoshiki['AC3'] = tempo4
solucoesFutoshiki['AC3'] = sol4
# Imprimir resultados
print(f"Tempo médio de execução: {tempo4:.6f}  s")
print('Assignment (FutoshikiComAC3) = ', sol4)

Tempo médio de execução: 0.004550  s
Assignment (FutoshikiComAC3) =  {(0, 0): 1, (0, 1): 7, (0, 2): 6, (0, 3): 4, (0, 4): 5, (0, 5): 2, (0, 6): 3, (1, 0): 2, (1, 1): 6, (1, 2): 4, (1, 3): 3, (1, 4): 1, (1, 5): 7, (1, 6): 5, (2, 0): 3, (2, 1): 2, (2, 2): 1, (2, 3): 5, (2, 4): 4, (2, 5): 6, (2, 6): 7, (3, 0): 7, (3, 1): 5, (3, 2): 2, (3, 3): 6, (3, 4): 3, (3, 5): 1, (3, 6): 4, (4, 0): 4, (4, 1): 3, (4, 2): 7, (4, 3): 1, (4, 4): 6, (4, 5): 5, (4, 6): 2, (5, 0): 6, (5, 1): 4, (5, 2): 5, (5, 3): 2, (5, 4): 7, (5, 5): 3, (5, 6): 1, (6, 0): 5, (6, 1): 1, (6, 2): 3, (6, 3): 7, (6, 4): 2, (6, 5): 4, (6, 6): 6}


- Backtracking com heurística MRV

In [51]:
# Resolver e guardar
sol5, tempo5 = search_time(backtracking_search, futoshiki7ext, select_unassigned_variable = mrv)
# Guarda resultados
temposFutoshiki['MRV'] = tempo5
solucoesFutoshiki['MRV'] = sol5
# Imprimir resultados
print(f"Tempo médio de execução: {tempo5:.6f} s")
print('Assignment (FutoshikiComHeuristica) = ', sol5)

Tempo médio de execução: 0.005948 s
Assignment (FutoshikiComHeuristica) =  {(6, 2): 3, (2, 3): 5, (2, 1): 2, (0, 6): 3, (5, 5): 3, (0, 2): 6, (0, 5): 2, (1, 3): 3, (4, 3): 1, (2, 4): 4, (6, 0): 5, (1, 6): 5, (5, 6): 1, (5, 1): 4, (6, 5): 4, (3, 6): 4, (3, 0): 7, (0, 3): 4, (3, 5): 1, (0, 0): 1, (6, 3): 7, (1, 0): 2, (5, 4): 7, (2, 5): 6, (1, 4): 1, (1, 2): 4, (5, 2): 5, (5, 3): 2, (6, 6): 6, (0, 4): 5, (4, 6): 2, (6, 1): 1, (3, 2): 2, (2, 6): 7, (3, 4): 3, (4, 0): 4, (2, 0): 3, (6, 4): 2, (2, 2): 1, (0, 1): 7, (3, 3): 6, (4, 4): 6, (5, 0): 6, (3, 1): 5, (4, 2): 7, (1, 1): 6, (4, 1): 3, (1, 5): 7, (4, 5): 5}


- Backtracking com Heurística MRV, Inferência Forward-Checking e pré-processamento AC3

In [54]:
futo_AC3 = AC3(futoshiki7ext)
# Resolver e guardar
sol6, tempo6 = search_time(backtracking_search, futo_AC3, select_unassigned_variable=mrv, inference=forward_checking)
# Guarda resultados
temposFutoshiki['AC3_MVR_MAC'] = tempo6
solucoesFutoshiki['AC3_MVR_MAC'] = sol6
# Imprimir resultados
print(f"Tempo médio de execução: {tempo6:.6f} s")
print('Assignment (FutoshikiHeuristicaInferenciaAC3) = ', sol6)

Tempo médio de execução: 0.010983 s
Assignment (FutoshikiHeuristicaInferenciaAC3) =  {(3, 2): 2, (0, 4): 5, (5, 1): 4, (4, 0): 4, (6, 6): 6, (6, 5): 4, (3, 6): 4, (1, 2): 4, (6, 3): 7, (5, 2): 5, (2, 4): 4, (0, 0): 1, (2, 6): 7, (4, 4): 6, (4, 1): 3, (1, 1): 6, (3, 1): 5, (4, 5): 5, (0, 2): 6, (3, 4): 3, (4, 6): 2, (3, 0): 7, (3, 5): 1, (1, 4): 1, (5, 6): 1, (6, 2): 3, (1, 5): 7, (5, 3): 2, (3, 3): 6, (1, 3): 3, (2, 0): 3, (4, 2): 7, (2, 1): 2, (6, 4): 2, (0, 6): 3, (0, 5): 2, (5, 4): 7, (2, 3): 5, (0, 3): 4, (1, 6): 5, (1, 0): 2, (0, 1): 7, (6, 1): 1, (2, 2): 1, (5, 5): 3, (4, 3): 1, (6, 0): 5, (2, 5): 6, (5, 0): 6}


In [55]:
temposSorted = dict(sorted(temposFutoshiki.items(), key=lambda x:x[1]))
pd.DataFrame(temposSorted, index=['Tempo médio de execução']).T

,Tempo médio de execução
FRWDchecking,0.004150
AC3,0.004550
MRV,0.005948
MAC,0.006409
AC3_MVR_MAC,0.010983
SemInferência,51.257967


In [56]:
pd.DataFrame(solucoesFutoshiki).T.drop_duplicates()

0                    1        ...  5        6                  
               0  1  2  3  4  5  6  0  1  2  ...  4  5  6  0  1  2  3  4  5  6
SemInferência  1  7  6  4  5  2  3  2  6  4  ...  7  3  1  5  1  3  7  2  4  6

[1 rows x 49 columns]

Para a nossa configuração encontramos que um problema 7x7 de dificuldade extrema do website indicado no enunciado do projeto, tem um tempo de execução por volta de 60 segundos. Portanto conseguimos resolver problemas 6x6 em menos de 1 minuto, e alguns problemas 7x7 mais simples em menos de um minuto.
<br>
Podemos ainda comparar os tempos de execução dos diferentes algoritmos. Como seria de esperar, o algoritmo sem inferência, sem heurística e sem pré-processamento demora muito mais tempo que os restantes, uma vez que este não tem nenhuma informação que lhe permita inferir um melhor percurso
<br>
Isto permite-nos concluir que a complexidade das restrições de desigualdade impostas no problema vão afetar bastante o tempo de procura.

Futoshiki 8x8

In [58]:
valores_iniciais8 = {(0,2) : [5], (0,4) : [8], (4,4) : [4], (0,6) : [2], (1,6) : [5], 
                     (0,7) : [7], (3,7) : [3]}

desigualdades8 = {((3,0),(4,0)) : maior, ((1,1),(1,0)) : maior, ((3,1),(4,1)) : menor, \
                  ((4,1),(4,0)) : menor, ((5,1),(5,0)) : maior, ((6,1),(7,1)) : menor, \
                  ((0,2),(1,2)) : menor, ((2,2),(2,1)) : menor, ((0,3),(0,2)) : menor, \
                  ((0,3),(1,3)) : maior, ((2,3),(3,3)) : maior, ((3,3),(3,2)) : menor, \
                  ((4,3),(5,3)) : menor, ((6,3),(7,3)) : menor, ((7,3),(7,2)) : menor, \
                  ((3,4),(4,4)) : menor, ((0,6),(0,5)) : maior, ((1,6),(1,5)) : menor, \
                  ((2,6),(3,6)) : maior, ((3,6),(3,5)) : maior, ((4,6),(5,6)) : maior, \
                  ((5,6),(5,5)) : maior, ((6,6),(7,6)) : menor, ((7,6),(7,5)) : menor, \
                  ((1,7),(1,6)) : maior, ((3,7),(4,7)) : maior, ((4,7),(5,7)) : menor, \
                  ((6,7),(7,7)) : menor}

In [59]:
futo8 = quadrado_latino(8, valores_iniciais8, desigualdades8)
f8_ac3 = AC3(futo8)
start = timeit.default_timer()
solution = backtracking_search(f8_ac3, inference=forward_checking)
end = timeit.default_timer()
print(solution)

{(0, 0): 3, (0, 1): 6, (0, 2): 5, (0, 3): 4, (0, 4): 8, (0, 5): 1, (0, 6): 2, (0, 7): 7, (1, 0): 1, (1, 1): 4, (1, 2): 6, (1, 3): 2, (1, 4): 3, (1, 5): 7, (1, 6): 5, (1, 7): 8, (2, 0): 4, (2, 1): 8, (2, 2): 2, (2, 3): 6, (2, 4): 5, (2, 5): 3, (2, 6): 7, (2, 7): 1, (3, 0): 8, (3, 1): 1, (3, 2): 7, (3, 3): 5, (3, 4): 2, (3, 5): 4, (3, 6): 6, (3, 7): 3, (4, 0): 7, (4, 1): 5, (4, 2): 3, (4, 3): 1, (4, 4): 4, (4, 5): 6, (4, 6): 8, (4, 7): 2, (5, 0): 5, (5, 1): 7, (5, 2): 1, (5, 3): 8, (5, 4): 6, (5, 5): 2, (5, 6): 3, (5, 7): 4, (6, 0): 6, (6, 1): 2, (6, 2): 4, (6, 3): 3, (6, 4): 7, (6, 5): 8, (6, 6): 1, (6, 7): 5, (7, 0): 2, (7, 1): 3, (7, 2): 8, (7, 3): 7, (7, 4): 1, (7, 5): 5, (7, 6): 4, (7, 7): 6}


In [61]:
visualizacao_futoshiki(8, futo8, desigualdades8, solution)

Estado Inicial:
0   0   0   0 > 0   0   0   0
    ∧           ∨   ∧        
0   0   0   0 < 0   0   0 < 0
        ∨                    
5 < 0   0   0   0   0   0   0
∨           ∨               ∨
0 > 0   0 > 0   0 < 0   0 < 0
                             
8   0   0   0 < 4   0   0   0
                             
0   0   0   0   0   0   0   0
∧   ∨       ∧       ∧       ∨
2   5   0 > 0   0 > 0   0 < 0
    ∧                        
7   0   0   3 > 0 < 0   0 < 0
                             

Solução:
3   1   4   8 > 7   5   6   2
    ∧           ∨   ∧        
6   4   8   1 < 5   7   2 < 3
        ∨                    
5 < 6   2   7   3   1   4   8
∨           ∨               ∨
4 > 2   6 > 5   1 < 8   3 < 7
                             
8   3   5   2 < 4   6   7   1
                             
1   7   3   4   6   2   8   5
∧   ∨       ∧       ∧       ∨
2   5   7 > 6   8 > 3   1 < 4
    ∧                        
7   8   1   3 > 2 < 4   5 < 6
                             



## Visualização do problema

In [60]:
def visualizacao_futoshiki(n, problema_csp, desigualdades, solucao):
    
    def mapeamento_2D_para_1D(x,y):
        return y*(n+(n-1)*3+1) + x

    def escrita_numeros(x,y):
        return mapeamento_2D_para_1D(x*4, y*2)
    
    def escrita_desigualdades_linha(x,y):
        return mapeamento_2D_para_1D(x*4+2, y*2)
    
    def escrita_desigualdades_coluna(x,y):
        return mapeamento_2D_para_1D(x*4, y*2-1)

    def escrever_desigualdade(operacao, direcao):
        if direcao == "x":
            if operacao.__name__ == "maior":
                return '>'
            else:
                return '<'
        else:
            if operacao.__name__ == "maior":
                return u"\u2227" # Carater ∧
            else:
                return u"\u2228" # Carater ∨

    board_inicial = ([" "] * (n+(n-1)*3) + ["\n"]) * (n*2)
    for var, dom in problema_csp.domains.items():
        if len(dom) == 1:
            board_inicial[escrita_numeros(var[0],var[1])] = str(dom[0])
        else:
            board_inicial[escrita_numeros(var[0],var[1])] = "0"
    for var, operacao in desigualdades.items():
        if (var[0][1] == var[1][1]):
            board_inicial[escrita_desigualdades_linha(var[0][0],var[0][1])] = escrever_desigualdade(operacao, "x")
        else:
            board_inicial[escrita_desigualdades_coluna(var[0][0],var[0][1])] = escrever_desigualdade(operacao, "y")
    
    board_final = board_inicial.copy()
    for var, valor in solucao.items():
        board_final[escrita_numeros(var[0],var[1])] = str(valor)
    

    print("Estado Inicial:")
    print(''.join(board_inicial))
    print("Solução:")
    print(''.join(board_final))

Podemos agora visualizar o problema futoshiki 5x5

In [62]:
visualizacao_futoshiki(5, futoshiki5, desigualdades5, backtracking_search(futoshiki5))

Estado Inicial:
0   0   0   0 > 0
                 
0 < 0   0   0   0
                 
0 < 0   0   0   0
            ∨    
2   0   0   0   3
        ∧       ∧
0   0 > 0   0   0
                 

Solução:
5   1   4   3 > 2
                 
3 < 4   5   2   1
                 
1 < 2   3   5   4
            ∨    
2   5   1   4   3
        ∧       ∧
4   3 > 2   1   5
                 



E o problema futoshiki 7x7

In [65]:
visualizacao_futoshiki(7, futoshiki7ext, desigualdades7ext, backtracking_search(futoshiki7ext))

Estado Inicial:
0   0 < 0 < 0   4   0 > 0
        ∨           ∨    
0 > 0 > 0 < 0   0 < 0   0
        ∨           ∧    
0 > 0   0   2   0   0   0
    ∨   ∧                
0 > 0   0 < 0   0 < 0   0
∧   ∨                   ∨
0   0   0   0 < 0 < 0   0
            ∨            
0   0   0   0   0   0   0
∧           ∧            
0   0   0   0   0   0   0
                         

Solução:
1   2 < 3 < 7   4   6 > 5
        ∨           ∨    
7 > 6 > 2 < 5   3 < 4   1
        ∨           ∧    
6 > 4   1   2   7   5   3
    ∨   ∧                
4 > 3   5 < 6   1 < 2   7
∧   ∨                   ∨
5   1   4   3 < 6 < 7   2
            ∨            
2   7   6   1   5   3   4
∧           ∧            
3   5   7   4   2   1   6
                         

